In [5]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 4.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import os

# Initialize the face detector using Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to extract frames and detect faces
def process_video(video_path, save_dir, video_name, max_frames=10):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_face_count = 0
    video_basename = os.path.splitext(video_name)[0]  # Extract video name without extension
    
    while cap.isOpened() and saved_face_count < max_frames:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame_count += 1
        if frame_count % int(cap.get(cv2.CAP_PROP_FRAME_COUNT) // max_frames) == 0:
            # Convert frame to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Detect faces in the frame
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            
            for (x, y, w, h) in faces:
                # Extract the face region from the frame
                face = frame[y:y+h, x:x+w]
                
                # Resize the face to a standard size (optional)
                face_resized = cv2.resize(face, (200, 200))
                
                # Save the face as a PNG image with a unique filename
                save_path = os.path.join(save_dir, f"{video_basename}_face_{saved_face_count + 1}.png")
                cv2.imwrite(save_path, face_resized)
                saved_face_count += 1
                
                if saved_face_count >= max_frames:
                    break
    
    cap.release()

# Path to the dataset folder
dataset_dir = "dataset"
output_dir = "output"

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through the classes (real and fake)
for class_name in ["real", "fake"]:
    class_dir = os.path.join(dataset_dir, class_name)
    output_class_dir = os.path.join(output_dir, class_name)
    
    # Ensure output class folder exists
    if not os.path.exists(output_class_dir):
        os.makedirs(output_class_dir)
    
    # Process each video in the class folder
    for video_name in os.listdir(class_dir):
        if video_name.endswith(".mp4"):
            video_path = os.path.join(class_dir, video_name)
            process_video(video_path, output_class_dir, video_name)
